In [3]:
import random

# Step 1: Read CSV manually
def read_csv_manual(filepath):
    with open(filepath, 'r') as file:
        lines = file.readlines()
    header = lines[0].strip().split(',')
    data = [line.strip().split(',') for line in lines[1:] if line.strip()]
    return header, data

# Step 2: Preprocess - feature conversion and label encoding
def preprocess_data(header, data):
    features = []
    labels = []
    label_map = {}
    label_counter = 0
    for row in data:
        features.append([float(x) for x in row[1:-1]])  # drop ID and label
        label = row[-1]
        if label not in label_map:
            label_map[label] = label_counter
            label_counter += 1
        labels.append(label_map[label])
    return features, labels, label_map

# Step 3: Standardize features
def standardize_features(features):
    n_samples = len(features)
    n_features = len(features[0])
    means = [sum(f[i] for f in features) / n_samples for i in range(n_features)]
    stds = [
        (sum((f[i] - means[i]) ** 2 for f in features) / n_samples) ** 0.5
        for i in range(n_features)
    ]
    scaled = [
        [(f[i] - means[i]) / stds[i] if stds[i] else 0 for i in range(n_features)]
        for f in features
    ]
    return scaled

# Step 4: Manual train-test split
def train_test_split_manual(X, y, test_size=0.2):
    combined = list(zip(X, y))
    random.seed(42)
    random.shuffle(combined)
    split_idx = int(len(combined) * (1 - test_size))
    train, test = combined[:split_idx], combined[split_idx:]
    X_train, y_train = zip(*train)
    X_test, y_test = zip(*test)
    return list(X_train), list(X_test), list(y_train), list(y_test)

# Step 5: Euclidean distance
def euclidean_distance(a, b):
    return sum((x - y) ** 2 for x, y in zip(a, b)) ** 0.5

# Step 6: KNN classifier
def knn_predict(X_train, y_train, x_test, k=5):
    distances = [(euclidean_distance(x_test, x), y) for x, y in zip(X_train, y_train)]
    distances.sort()
    top_k = [label for _, label in distances[:k]]
    return max(set(top_k), key=top_k.count)

# Step 7: Accuracy score
def calculate_accuracy(y_true, y_pred):
    correct = sum(1 for yt, yp in zip(y_true, y_pred) if yt == yp)
    return correct / len(y_true)

# Step 8: Confusion matrix
def compute_confusion_matrix(y_true, y_pred, num_classes):
    matrix = [[0 for _ in range(num_classes)] for _ in range(num_classes)]
    for t, p in zip(y_true, y_pred):
        matrix[t][p] += 1
    return matrix

def print_classification_report(conf_matrix, label_map):
    num_classes = len(label_map)
    print("\nClassification Report:")
    print(f"{'Class':<15}{'Precision':<10}{'Recall':<10}{'F1-Score':<10}{'Support':<10}")
    print("-" * 55)
    
    for i in range(num_classes):
        class_name = [k for k, v in label_map.items() if v == i][0]
        tp = conf_matrix[i][i]
        fp = sum(conf_matrix[j][i] for j in range(num_classes) if j != i)
        fn = sum(conf_matrix[i][j] for j in range(num_classes) if j != i)
        support = sum(conf_matrix[i])  # total actual instances of class i

        precision = tp / (tp + fp) if (tp + fp) != 0 else 0.0
        recall = tp / (tp + fn) if (tp + fn) != 0 else 0.0
        f1 = 2 * precision * recall / (precision + recall) if (precision + recall) != 0 else 0.0

        print(f"{class_name:<15}{precision:<10.2f}{recall:<10.2f}{f1:<10.2f}{support:<10}")


# Main execution
def main():
    filepath = 'Iris.csv'  # make sure this file is in the same directory
    header, raw_data = read_csv_manual(filepath)
    features, labels, label_map = preprocess_data(header, raw_data)
    scaled_features = standardize_features(features)
    X_train, X_test, y_train, y_test = train_test_split_manual(scaled_features, labels)

    y_pred = [knn_predict(X_train, y_train, x, k=5) for x in X_test]
    accuracy = calculate_accuracy(y_test, y_pred)
    conf_matrix = compute_confusion_matrix(y_test, y_pred, len(label_map))

    # Reverse map for label display
    reverse_map = {v: k for k, v in label_map.items()}

    print("\n--- KNN Classifier Results (No Libraries Used) ---")
    print(f"Accuracy: {accuracy * 100:.2f}%")
    print("\nConfusion Matrix:")
    for row in conf_matrix:
        print(row)
    print("\nLabel Mapping:")
    for label, idx in label_map.items():
        print(f"{idx} = {label}")
        
    print_classification_report(conf_matrix, label_map)


if __name__ == "__main__":
    main()



--- KNN Classifier Results (No Libraries Used) ---
Accuracy: 96.67%

Confusion Matrix:
[11, 0, 0]
[0, 10, 1]
[0, 0, 8]

Label Mapping:
0 = Iris-setosa
1 = Iris-versicolor
2 = Iris-virginica

Classification Report:
Class          Precision Recall    F1-Score  Support   
-------------------------------------------------------
Iris-setosa    1.00      1.00      1.00      11        
Iris-versicolor1.00      0.91      0.95      11        
Iris-virginica 0.89      1.00      0.94      8         
